In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing

Считываем данные:

In [2]:
train = pd.read_csv('Desktop/ML/data/Task4/train.csv', sep = '\t')
test = pd.read_csv('Desktop/ML/data/Task4/test.csv')

In [3]:
print(train.shape)
print(test.shape)

(237969, 386)
(4045, 387)


In [4]:
train.drop('DSSR', axis=1, inplace=True)
test.drop('DSSR', axis=1, inplace=True)

In [5]:
test.drop('Id', axis=1, inplace=True)
test.drop('index', axis=1, inplace=True)

In [6]:
train = train.dropna()

In [7]:
print(train.shape)
print(test.shape)

(209043, 385)
(4045, 384)


Кодируем данные:

In [8]:
columns = train["pdb_chain"].copy()
columns = columns.append(test["pdb_chain"])
encoders = preprocessing.LabelEncoder()
encoders.fit(columns)
train["pdb_chain"] = encoders.transform(train["pdb_chain"])
test["pdb_chain"] = encoders.transform(test["pdb_chain"])

In [9]:
y_train = train["mg"]
X_train = train.drop('mg', axis=1, inplace=False)
X_test = test 

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(209043, 384)
(209043,)
(4045, 384)


In [11]:
X = np.vstack((X_train, X_test))

In [12]:
X = preprocessing.scale(X, with_mean=True, with_std=True)

In [13]:
X_train, X_test = X[:X_train.shape[0]], X[X_train.shape[0]:]

In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(209043, 384)
(209043,)
(4045, 384)


In [15]:
def submit(preds):
    submission = pd.read_csv("Desktop/ML/data/Task4/sample_submission.csv")
    submission["mg"] = preds
    submission.to_csv("Desktop/ML/data/Task4/submission3.csv", index=False)

In [16]:
clf = xgb.XGBClassifier(max_depth=3, base_score=0.5, learning_rate=0.2, booster='gbtree',
                        n_estimators=3000, n_jobs=4, colsample_bylevel=1, gamma=0,
                        colsample_bytree=X_train.shape[1]**(1/2)/X_train.shape[1],
                        min_child_weight=1, missing=None, max_delta_step=0,
                        objective='binary:logistic', random_state=0, reg_alpha=0.005)

In [17]:
%%time
clf.fit(X_train, y_train)

CPU times: user 14min 56s, sys: 2.46 s, total: 14min 59s
Wall time: 3min 46s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.05103103630798287, gamma=0, learning_rate=0.2,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=3000, n_jobs=4, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0.005,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [18]:
preds = clf.predict(X_test)

/Users/apple/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
submit(preds)

In [20]:
!head Desktop/ML/data/Task4/submission3.csv

Id,mg
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
